##### Load modules

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

##### Inference on test data

In [2]:
df = pd.read_pickle('test.pickle')

In [3]:
def batch_inference(models, df, n_batch):
    
    result = []
    n_batch = int(n_batch)
    
    for idx in np.arange(0, df.shape[0], n_batch):
        progress = idx / df.shape[0] * 100
        progress = np.round(progress, 2)
        print('\r', progress, end='')
        
        start = idx
        end = (idx + n_batch)
        batch = df[start:end]
                
        pred = [model.predict(batch) for model in models]
        pred = np.mean(pred, axis=0)
        
        result.append(pred)
        
    print('\r', '100.00')
    result = np.concatenate(result)
    return result

In [4]:
result = []

for meter in range(4):
    print('========== Inference for {} meter dataset =========='.format(meter))
    
    # Load models for given meter
    models = [lgb.Booster(model_file='models/{}_{}.model'.format(meter, i)) for i in range(4)]
    
    # Prepare features
    where = df.meter == meter
    X = df[where].drop(columns='row_id')
    y = df.loc[where, ['row_id']]
    
    # Conduct inference and store hypothesis
    y['meter_reading'] = batch_inference(models, X, 1e4)
    result.append(y)

========== Inference for 0 meter dataset ==========
 100.00
========== Inference for 1 meter dataset ==========
 100.00
========== Inference for 2 meter dataset ==========
 100.00
========== Inference for 3 meter dataset ==========
 100.00


In [5]:
df = pd.concat(result)
df['meter_reading'] = np.clip(df.meter_reading, 0, None)
df['meter_reading'] = np.expm1(df.meter_reading)
df.sort_values('row_id', inplace=True)
df.to_csv('submission.csv', index=False)